### Analisi Dati Facebook

1. pip install facebook-sdk: https://github.com/mobolic/facebook-sdk, https://facebook-sdk.readthedocs.io/en/latest/
    
2. Tool di Esplorazioni API Graph: https://developers.facebook.com/tools/explorer/145634995501895/#_=_
    
3. ID Pagina Facebook: http://findmyfbid.com/

In [4]:
import facebook
import os
import requests
import pandas as pd

# Leggo Configuration File
config = {}
config_path = os.path.join(os.environ['HOME'], 'opendata')
config_name = 'config.py'
config_file = os.path.join(config_path,config_name)
exec(open(config_file).read(),config)
token=config['TOKEN_FB']

# Utilizzo token generoato manualmente con la mia utenza sul Tool di Esplorazioni API Graph
graph = facebook.GraphAPI(access_token=token, version='2.7')

In [5]:
# TEST

# Esempio di chiamata su Tool Esplorazione: me?fields=id,name -> Result è un Dict (casomai Dataframe Pandas)
info = graph.get_object(id='me')
info

{'id': '10153605453803372', 'name': 'Andrea Cesarini'}

In [6]:
# PAGINA FACEBOOK

# Recupero ID della Pagina di Salvini (https://www.facebook.com/salviniofficial/?fref=ts) da findmyfbid 
# Esempio di chiamata: 252306033154?fields=id,name,birthday
info_s = graph.get_object(id='252306033154', fields='name,fan_count,posts')

In [7]:
# NUMERO FAN

# Recupero il numero di Fan (sono quelli che mettono mi piace, non è detto che siano quelli che lo seguono in
# quanto uno mette mi piace anche solo per seguirlo)
num_fan = info_s['fan_count']
num_fan

1696884

# 1. Distribuzione likes ai posts in base alla data dei post (aggregati per anno / mese)
## Crescita social delle pagine politiche -> In base alla data, legarle ad avvenimenti

In [8]:
# Recupero TUTTI i posts sulla pagina di Salvini
posts = graph.get_connections(id='252306033154', connection_name='posts')
all_posts = {}
while(True):
    try:
        for post in posts['data']:
            all_posts[post['id']]=post['created_time']
        posts=requests.get(posts['paging']['next']).json()
    except KeyError:
        break
len(all_posts)

16007

In [ ]:
#all_posts

In [9]:
df = pd.DataFrame(list(all_posts.items()), columns=['ID', 'Post_Date'])

In [10]:
df.head(5)

,ID,Post_Date
0,252306033154_10151694974523155,2013-09-26T17:54:20+0000
1,252306033154_10152854711393155,2015-03-29T10:07:10+0000
2,252306033154_10151493392598155,2013-06-13T08:15:41+0000
3,252306033154_10151156433468155,2012-12-11T21:57:01+0000
4,252306033154_10152607210623155,2014-12-21T12:42:02+0000


In [12]:
# Salvo il DF per non ripetere la Chiamata
dir_df = os.path.join(os.path.abspath(''),'stg')
df_filename = r'df_posts.pkl'
df_fullpath = os.path.join(dir_df, df_filename)
df.to_pickle(df_fullpath)